In [2]:
import pickle

# Load the model from the file
with open('random_forest_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
    

print("Model loaded successfully!")


Model loaded successfully!


In [3]:
# Test the model manually with a malicious record

malicious_record = [
    [6000, 3500.000, 570.000000, 2900.000000, 580, 1540.000000, 600.00, 450.000000, 320.000, 150.000]
]
predictions = loaded_model.predict(malicious_record)
# predictions

In [ ]:
from scapy.all import sniff, get_if_list, conf
import numpy as np
import pickle
import time
import threading

with open('random_forest_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

packets_captured = []  

def extract_and_predict():
    if len(packets_captured) < 2: 
        print("Not enough packets captured for feature extraction.")
        packets_captured.clear()
        return

    flow_duration = packets_captured[-1].time - packets_captured[0].time if packets_captured else 0
    packet_lengths = [len(pkt) for pkt in packets_captured]
    packet_times = [pkt.time for pkt in packets_captured]

    bwd_packet_length_mean = np.mean(packet_lengths)
    bwd_packet_length_std = np.std(packet_lengths)
    flow_iat_std = np.std(np.diff(packet_times)) if len(packet_times) > 1 else 0
    fwd_iat_total = np.sum(np.diff(packet_times)) if len(packet_times) > 1 else 0
    fwd_iat_std = np.std(np.diff(packet_times)) if len(packet_times) > 1 else 0
    packet_length_mean = np.mean(packet_lengths)
    packet_length_std = np.std(packet_lengths)
    avg_bwd_segment_size = np.mean(packet_lengths)
    idle_mean = np.mean(np.diff(packet_times)) if len(packet_times) > 1 else 0

    features = [
        flow_duration,
        bwd_packet_length_mean,
        bwd_packet_length_std,
        flow_iat_std,
        fwd_iat_total,
        fwd_iat_std,
        packet_length_mean,
        packet_length_std,
        avg_bwd_segment_size,
        idle_mean
    ]

    print("\n")
    print("Extracted features:", features)

    features_array = np.array([features])

    prediction = loaded_model.predict(features_array)[0]
    print(f"Prediction: {'Malicious' if prediction == 1 else 'Safe'}")

    packets_captured.clear()

def capture_packets(interface):
    print(f"Starting packet capture on {interface}...")

    def process_packet(packet):
        packets_captured.append(packet)

    while True:
        sniff(
            iface=interface,
            prn=process_packet,
            store=0,
            filter="ip",  
            timeout=5  
        )
        extract_and_predict()

interfaces = get_if_list()
print("Available network interfaces:")
for i, iface in enumerate(interfaces):
    print(f"{i}: {iface}")

selected_index = int(input("Select a network interface index (0-{}): ".format(len(interfaces) - 1)))
selected_interface = interfaces[selected_index]

capture_thread = threading.Thread(target=capture_packets, args=(selected_interface,))
capture_thread.start()


ping -t -l 60000 < IPV4 ADDRESS>              use this in cmd to ping ,,,to take out the ip configuration use ip config in cmd